In [1]:
import os
import geopandas as gpd

# os.environ["USE_PYGEOS"] = os.environ.get("USE_PYGEOS", "0")
local_crs = 32636
# path to data
example_data_path = "./data"

In [2]:
from masterplan_tools.preprocessing import DataGetter, AggregateParameters

blocks = gpd.read_parquet(os.path.join(example_data_path, "blocks_cutter_result.parquet")).to_crs(local_crs)
getter = DataGetter(blocks=blocks)

In [3]:
buildings = gpd.read_parquet(os.path.join(example_data_path, "buildings.parquet"))
greenings = gpd.read_parquet(os.path.join(example_data_path, "greenings.parquet")).rename_geometry('geometry')
parkings = gpd.read_parquet(os.path.join(example_data_path, "parkings.parquet")).rename_geometry('geometry')

aggr_params = AggregateParameters(
  buildings=buildings,
  greenings=greenings,
  parkings=parkings
)

aggregated_blocks = getter.aggregate_blocks_info(params=aggr_params)
aggregated_blocks.to_gdf()

Restoring living area:   0%|          | 0/3748 [00:00<?, ?it/s]

Restoring living area squash:   0%|          | 0/3748 [00:00<?, ?it/s]

,geometry,landuse,block_id,is_living,current_population,floors,current_living_area,current_green_capacity,current_green_area,current_parking_capacity,current_industrial_area,area
0,"POLYGON ((344943.719 6647610.514, 344959.205 6...",buildings,0,False,0.0,NaN,0.0,0.0,0.0,0.0,0.0000,4.113091e+01
1,"POLYGON ((344968.687 6647544.000, 344971.625 6...",buildings,1,False,0.0,NaN,0.0,0.0,0.0,0.0,0.0000,1.622221e+02
2,"POLYGON ((344929.592 6647257.521, 344931.964 6...",buildings,2,False,0.0,0.0,0.0,0.0,0.0,0.0,42.6397,4.864831e+01
3,"POLYGON ((344840.676 6647432.974, 344853.943 6...",buildings,3,False,0.0,NaN,0.0,0.0,0.0,0.0,0.0000,6.148153e+01
4,"POLYGON ((344856.398 6647391.171, 344900.097 6...",buildings,4,False,0.0,NaN,0.0,0.0,0.0,0.0,0.0000,3.897924e+02
...,...,...,...,...,...,...,...,...,...,...,...,...
1349,"POLYGON ((346243.206 6647701.284, 346243.206 6...",selected_area,1349,False,0.0,NaN,0.0,0.0,0.0,0.0,0.0000,4.424234e-10
1350,"POLYGON ((347483.986 6647345.029, 347483.627 6...",selected_area,1350,False,0.0,NaN,0.0,0.0,0.0,0.0,0.0000,1.971091e-11
1351,"POLYGON ((347463.977 6647381.345, 347456.841 6...",selected_area,1351,False,0.0,NaN,0.0,0.0,0.0,0.0,0.0000,7.996164e-10
1352,"POLYGON ((344028.826 6648834.242, 344112.884 6...",no_dev_area,1352,False,0.0,3.0,0.0,0.0,0.0,0.0,1535.7700,8.966580e+04


In [4]:
import networkx as nx

#we get this from CityGeoTools
transport_graph = nx.read_graphml(os.path.join(example_data_path, "new_graph.graphml"))

accessibility_matrix = getter.get_accessibility_matrix(transport_graph)

In [5]:
accessibility_matrix.df

id,0,1,2,3,4,5,6,7,8,9,...,1344,1345,1346,1347,1348,1349,1350,1351,1352,1353
id,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.1,8.2,0.1,8.2,8.2,5.5,3.9,3.9,13.2,...,22.2,19.0,17.4,17.4,16.7,19.4,19.5,20.3,20.1,19.2
1,0.1,0.0,8.1,0.0,8.1,8.1,5.6,4.0,4.0,13.3,...,22.3,19.1,17.5,17.5,16.8,19.5,19.6,20.4,20.2,19.3
2,8.2,8.1,0.0,8.1,0.0,0.0,13.6,11.6,11.6,9.0,...,22.6,19.4,18.3,18.3,18.0,22.5,19.5,20.3,22.4,20.4
3,0.1,0.0,8.1,0.0,8.1,8.1,5.6,4.0,4.0,13.3,...,22.3,19.1,17.5,17.5,16.8,19.5,19.6,20.4,20.2,19.3
4,8.2,8.1,0.0,8.1,0.0,0.0,13.6,11.6,11.6,9.0,...,22.6,19.4,18.3,18.3,18.0,22.5,19.5,20.3,22.4,20.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1349,20.1,20.2,21.6,20.2,21.6,21.6,15.1,17.1,17.1,18.0,...,17.9,14.7,13.6,13.6,13.3,0.0,11.8,12.6,19.0,15.7
1350,18.2,18.1,17.1,18.1,17.1,17.1,15.3,17.3,17.3,18.2,...,17.6,14.9,13.8,13.8,13.5,12.8,0.0,0.8,19.2,15.9
1351,18.0,17.9,16.9,17.9,16.9,16.9,15.1,17.1,17.1,18.0,...,16.8,14.7,13.6,13.6,13.3,13.6,0.8,0.0,19.0,15.7


In [6]:
aggregated_blocks.to_gdf().to_parquet(os.path.join(example_data_path, "data_getter_blocks.parquet"))
accessibility_matrix.df.to_pickle(os.path.join(example_data_path, "data_getter_matrix.pickle"))